In [86]:
import gurobipy as gp
from gurobipy import GRB

## Problema de dimensionamento de lotes capacitado

Considere a formulação do problema da dimensionamento de lotes capacitado.

$
\begin{align*}
\min \ & \sum_{t=1}^{n} p_{t} x_{t} + \sum_{t=1}^{n} h_t s_t + \sum_{t=1}^{n} f_t y_t \\
& s_{t-1} + x_{t} = d_{t} + s_{t} \ \mbox{ para } t=1, \ldots, n \\
& x_{t} \leq C \cdot y_t \mbox{ para } t=1, \ldots n \\
& s_{0} = s_{n} = 0 \\
& s \in \mathbb{R}_{+}^{n+1} \\
& x \in \mathbb{R}_{+}^{n} \\
& y \in \mathbb{B}^{n}
\end{align*}
$

Considere a instância a seguir com 4 períodos.

In [87]:
# demanda
d = [2, 4, 5, 1]

# custo de produção
p = [3, 3, 3, 3]

# capacidade
C = 4

# custo de estoque
h = [1, 2, 1, 1]

# custo fixo de cada fábrica
f = [12, 20, 16, 8]

# quantidade de períodos
N = range(len(d))

Considere a desigualdade
$$
x_t \leq d_t y_t \mbox{ para } t=1, \ldots, n
$$
e a desigualdade
$$
\sum_{i=1}^{t} y_{i} \geq \left\lceil  \dfrac{\sum_{i=1}^{t} d_{i}}{C} \right\rceil \mbox{ para } t=1, \ldots, n
$$
as quais são desigualdades válidas para o problema o problema de dimensionamento de lotes capacitado.

Implemente um algoritmo de plano de cortes usando estas duas desigualdades.

In [82]:
# criando modelo
model = gp.Model("cls")

# variáveis e função objetivo
y = model.addVars(N,vtype=GRB.BINARY,obj=f,name="y")
x = model.addVars(N, obj=p, name="x")
s = model.addVars(N, obj=h, name="s")

# sentido da função objetivo
model.modelSense = GRB.MINIMIZE

# restrições
for t in N:
    if t == 0:
        model.addConstr((x[t] == d[t] + s[t]))
    else:
        model.addConstr((s[t-1] + x[t] == d[t] + s[t]))

model.addConstrs((x[t] <=  C * y[t] for t in N))

s[0] = 0

s[len(d)] = 0

# salva o modelo
model.write('uls.lp')

In [83]:
# resolve o problema
model.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 8 rows, 12 columns and 19 nonzeros
Model fingerprint: 0x2903916b
Variable types: 8 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [1e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Presolve removed 8 rows and 12 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 1: 91 

Optimal solution found (tolerance 1.00e-04)
Best objective 9.100000000000e+01, best bound 9.100000000000e+01, gap 0.0000%


In [79]:
# imprimindo a solução ótima
print('Obj = %g' % model.objVal)

# imprimindo as soluções
for v in model.getVars():
    print('%s = %g' % (v.varName, v.x))

Obj = 91
y[0] = 1
y[1] = 1
y[2] = 1
y[3] = 0
x[0] = 4
x[1] = 4
x[2] = 4
x[3] = 0
s[0] = 2
s[1] = 2
s[2] = 1
s[3] = 0


In [80]:
# relaxando a variáveis inteiras do MIP
for v in model.getVars():
    if v.vType != GRB.CONTINUOUS:
        v.vType = GRB.CONTINUOUS

model.update()

# resolve o problema
model.optimize()

# imprime a solução ótima
for v in model.getVars():
    print(v.varName,'=', v.x)
    
# imprime o valor ótimo
print('opt =', model.objVal)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 8 rows, 12 columns and 19 nonzeros
Model fingerprint: 0xeb104cbf
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [1e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Presolve removed 6 rows and 8 columns
Presolve time: 0.01s
Presolved: 2 rows, 4 columns, 5 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.9930000e+01   5.035000e-01   0.000000e+00      0s
       3    8.5000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.01 seconds
Optimal objective  8.500000000e+01
y[0] = 1.0
y[1] = 0.75
y[2] = 1.0
y[3] = 0.25
x[0] = 4.0
x[1] = 3.0
x[2] = 4.0
x[3] = 1.0
s[0] = 2.0
s[1] = 1.0
s[2] = 0.0
s[3] = 0.0
opt = 85.0
